In [1]:
import pandas as pd
import json
from haversine import haversine
import datetime
today_str=str(datetime.datetime.now().date())

In [2]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))

In [3]:
demo_df=pd.read_csv("/home/jian/Docs/Household_and_Population/2016/Demo_Dataset.csv",dtype=str)
demo_df=demo_df[['ZIP_CODE','HH15','Estimate; Female: - 25 to 29 years','Estimate; Female: - 30 to 34 years',
                 'Estimate; Female: - 35 to 39 years','Estimate; Female: - 40 to 44 years',
                 'Estimate; Female: - 45 to 49 years','Estimate; Female: - 50 to 54 years']]
for col in demo_df.columns.tolist()[1:]:
    demo_df[col]=demo_df[col].astype(float)
demo_df['Female_25_54']=demo_df[['Estimate; Female: - 25 to 29 years','Estimate; Female: - 30 to 34 years',
                 'Estimate; Female: - 35 to 39 years','Estimate; Female: - 40 to 44 years',
                 'Estimate; Female: - 45 to 49 years','Estimate; Female: - 50 to 54 years']].sum(axis=1)
demo_df=demo_df[['ZIP_CODE','HH15','Female_25_54']].rename(columns={"ZIP_CODE":"zip_cd","HH15":"Households"})
demo_df['zip_cd']=demo_df['zip_cd'].apply(lambda x:x.zfill(5))

# 10_Miles

In [29]:
def finding_10_miles_zips(store_center,location_id,zip_for_store):
    result=pd.DataFrame()
    i=0
    for zip_5 in zip_centers.keys():    
        dist=haversine(store_center,zip_centers[zip_5],miles=True)
        if dist<=10:        
            df=pd.DataFrame({"zip_cd":zip_5,"Center_Lat":zip_centers[zip_5][0],"Center_Long":zip_centers[zip_5][1],"Distance_Mile":dist},index=[i])
            i=i+1
            result=result.append(df)
    result['Store_id']=store_location_id
    result['Store_zip']=zip_for_store
    result=result[['Store_id','Store_zip','zip_cd','Center_Lat','Center_Long','Distance_Mile']]
    result=pd.merge(result,demo_df,on="zip_cd",how="left")
    return result

In [30]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/New_TA/Zips_within_10_miles/BL_Zips_in_10_miles_"+today_str+"_Denver_12_stores.xlsx",engine="xlsxwriter")

In [31]:
store_list=pd.read_table("/home/jian/BigLots/static_files/MediaStormStores_20180703.txt",dtype=str,sep="|")
store_list['location_id']=store_list['location_id'].astype(int)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['zip_cd']=store_list['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
store_list=store_list[['location_id','zip_cd','latitude_meas','longitude_meas']]

In [32]:
Denver_Target_Store_List=[1914, 1920, 4117, 4133, 4513, 4586, 4121, 4341, 4486, 4490, 4517, 4572]
Devnver_Store_Df=store_list[store_list['location_id'].isin(Denver_Target_Store_List)].reset_index()
del Devnver_Store_Df['index']

In [33]:
'''
df_store_zips=pd.DataFrame()
for i in range(len(Devnver_Store_Df)):
    store_center=(Devnver_Store_Df['latitude_meas'][i],Devnver_Store_Df['longitude_meas'][i])
    zip_for_store=Devnver_Store_Df['zip_cd'][i]
    store_location_id=Devnver_Store_Df['location_id'][i]
    df=finding_10_miles_zips(store_center,store_location_id,zip_for_store)
    df_store_zips=df_store_zips.append(df)
df_store_zips['DMA']='Denver, CO'
df_target_zips_unique=df_store_zips[['zip_cd','DMA']].drop_duplicates()
df_target_zips_unique.to_excel(writer,"target_zips_unique",index=False)
df_store_zips.to_excel(writer,"12_store_zips",index=False)
writer.save()
'''

# P_S

In [18]:
store_level_TA_zips=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)
store_level_TA_zips['location_id']=store_level_TA_zips['location_id'].astype(int)

store_list=pd.read_table("/home/jian/BigLots/static_files/MediaStormStores_20180703.txt",dtype=str,sep="|")
store_list['location_id']=store_list['location_id'].astype(int)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['zip_cd']=store_list['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
store_list=store_list[['location_id','location_desc','address_line_1','address_line_2','zip_cd','latitude_meas','longitude_meas']]

In [32]:

df_P_S['store_lat']=store_list[store_list['location_id']==store]['latitude_meas'].unique()[0]

In [33]:
def extract_store_level_zip(store):
    df=store_level_TA_zips[store_level_TA_zips['location_id']==store]
    df_P=df[df['revenue_flag']=="P"]
    df_S=df[df['revenue_flag']=="S"]
    df_P_S=df_P.append(df_S)
    df_P_S['zip_cd']=df_P_S['zip'].apply(lambda x: x.zfill(5))
    df_P_S=df_P_S[['location_id','zip_cd','revenue_flag']]
    df_P_S['store_lat']=store_list[store_list['location_id']==store]['latitude_meas'].unique()[0]
    df_P_S['store_long']=store_list[store_list['location_id']==store]['longitude_meas'].unique()[0]
    df_P_S['location_desc']=store_list[store_list['location_id']==store]['location_desc'].unique()[0]
    df_P_S['address_line_1']=store_list[store_list['location_id']==store]['address_line_1'].unique()[0]
    df_P_S['address_line_2']=store_list[store_list['location_id']==store]['address_line_2'].unique()[0]
    store_lat=store_list[store_list['location_id']==store]['latitude_meas'].unique()[0]
    store_long=store_list[store_list['location_id']==store]['longitude_meas'].unique()[0]
    df_P_S['distance']=df_P_S['zip_cd'].apply(lambda x: haversine((store_lat,store_long),zip_centers[x],miles=True))
    
    return df_P_S
    
    

In [39]:
df_1629=extract_store_level_zip(1629)
df_1762=extract_store_level_zip(1762)
output=df_1629.append(df_1762)

In [40]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/New_TA/Zips_within_10_miles/BL_Zips_in_P_S_store_level_"+today_str+"_2_stores_1629_1762.xlsx",engine="xlsxwriter")
output.to_excel(writer,"1629_and_1762",index=False)
writer.save()

In [45]:
store_level_TA_zips[store_level_TA_zips['location_id']==1762].groupby(['revenue_flag'])['zip'].count().to_frame().reset_index()

,revenue_flag,zip
0,P,5
1,S,6
2,T,624


In [65]:
P_store_level_TA_zips_Devner_12_Stores.head(2)
unique_P_zips=P_store_level_TA_zips_Devner_12_Stores['zip'].unique().tolist()
unique_S_zips=[x for x in S_store_level_TA_zips_Devner_12_Stores['zip'].unique().tolist() if x not in unique_P_zips]
unique_P_df=pd.DataFrame({"zip_cd":unique_P_zips,"revenue_flag":"P"},index=range(len(unique_P_zips)))
unique_S_df=pd.DataFrame({"zip_cd":unique_S_zips,"revenue_flag":"S"},index=range(len(unique_S_zips)))


In [66]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/New_TA/Zips_within_10_miles/BL_Zips_in_P_S_store_level_"+today_str+"_Denver_12_stores.xlsx",engine="xlsxwriter")
unique_P_df.to_excel(writer,'unique_P_list',index=False)
unique_S_df.to_excel(writer,'unique_S_list',index=False)
P_store_level_TA_zips_Devner_12_Stores.to_excel(writer,"P_zips_by_store",index=False)
S_store_level_TA_zips_Devner_12_Stores.to_excel(writer,"S_zips_by_store",index=False)
writer.save()